### SECCIÓN: Validación del Diseño Experimental

In [4]:
import pandas as pd

df = pd.read_csv("Datos procesados/df_model_ready.csv")

print(df.shape)
print(df.columns)
print(df["experiment_group"].value_counts(dropna=False))


(70609, 14)
Index(['client_id', 'age_group', 'gender', 'tenure_months', 'num_accounts',
       'balance', 'balance_log', 'logins_6m', 'calls_6m', 'experiment_group',
       'web_events', 'web_visits', 'web_steps', 'web_confirms'],
      dtype='object')
experiment_group
Test             26968
Control          23532
No_experiment    20109
Name: count, dtype: int64


In [ ]:
# filtramos solo test y control
df_ab = df[df["experiment_group"].isin(["Test", "Control"])].copy()

print(df_ab["experiment_group"].value_counts())
print(df_ab.shape)

# Aseguramos q no haya contaminación con usuarios fuera del experimento.
# y q el análisis metodológico sea limpio.

experiment_group
Test       26968
Control    23532
Name: count, dtype: int64
(50500, 14)


-Test → 26.968
-Control → 23.532
-👉 Diferencia de tamaño ≈ 3.400 usuarios


In [9]:
# Tabla descriptiva de balance
balance_table = (df_ab
                 .groupby("experiment_group")
                 [["tenure_months","balance","logins_6m","calls_6m","num_accounts"]]
                 .agg(["mean", "std", "median"]))

balance_table


tenure_months                          balance  \
                          mean        std median           mean   
experiment_group                                                  
Control             151.060186  82.126443  137.0  150147.326660   
Test                149.852899  81.783390  134.0  148962.605032   

                                          logins_6m                   \
                            std    median      mean       std median   
experiment_group                                                       
Control           295806.950360  66024.18  6.166277  2.173014    6.0   
Test              307373.713775  65468.36  6.101851  2.177119    6.0   

                  calls_6m                  num_accounts                   
                      mean       std median         mean       std median  
experiment_group                                                           
Control           3.129213  2.184502    3.0     2.259914  0.538559    2.0  
Test              3.061941  2.190590    3.0     2.249917  0.529335    2.0

In [10]:
# Confirmación estadística del balance
from scipy.stats import ttest_ind

control = df_ab[df_ab["experiment_group"]=="Control"]
test = df_ab[df_ab["experiment_group"]=="Test"]

vars_to_check = ["tenure_months","balance","logins_6m","calls_6m","num_accounts"]

results = []
for v in vars_to_check:
    stat, p = ttest_ind(control[v], test[v], equal_var=False, nan_policy="omit")
    results.append((v, p))

results


[('tenure_months', np.float64(0.09875806956295248)),
 ('balance', np.float64(0.6593584865878536)),
 ('logins_6m', np.float64(0.0008999140061860009)),
 ('calls_6m', np.float64(0.0005667644591188616)),
 ('num_accounts', np.float64(0.0359623453959017))]

### tenure
- 👉Esto es prácticamente igual.
- No hay señal de que un grupo tenga clientes más antiguos.

- en p_value - No hay diferencia real en antigüedad.
### balance
- Eso es menos del 1%, las medianas también son muy parecidas (~66k vs ~65k).
- 👉No hay desbalance financiero relevante

- en p_value - Balance financiero bien equilibrado.
### Logins_6m
- irrelevante

- en p_value - Estadísticamente significativo - Significancia estadística ≠ relevancia práctica
### Calls_6m
- diferencia minima

- en p_value - Es estadísticamente significativa por el tamaño muestral, pero no es operacionalmente relevante
### Num_accounts
- practicamente identico

- en p_value - Significativo técnicamente - Pero la diferencia media era: 2.25 vs 2.24

### Conclusión técnica
- 👉Los grupos están extremadamente bien balanceados.
- No hay ninguna variable con diferencia llamativa.
- Eso sugiere:
. ✔️ Buena aleatorización
. ✔️ Alta validez interna

### p_value 
Metodológicamente:
- No hay desbalance relevante en variables estructurales fuertes (balance, tenure).
- Existen pequeñas diferencias en variables de actividad (logins, calls), pero de magnitud mínima.
- Las diferencias detectadas parecen ser efecto del gran tamaño muestral más que de un sesgo real
### el experimento presenta un adecuado balance entre grupos.

In [11]:
## 2. Posible sesgo por exclusión de usuarios con baja actividad
df_ab["logins_6m"].describe()


count    50488.000000
mean         6.131873
std          2.175423
min          3.000000
25%          4.000000
50%          6.000000
75%          8.000000
max          9.000000
Name: logins_6m, dtype: float64

In [12]:
df_ab["logins_6m"].value_counts().sort_index().head(10)


logins_6m
3.0     7311
4.0     7855
5.0     7283
6.0     5965
7.0     4914
8.0     4810
9.0    12350
Name: count, dtype: int64

el dataset final incluye únicamente usuarios con 3 o más logins en los últimos 6 meses, lo que indica que el experimento fue aplicado sobre una población digitalmente activa. Esto podría limitar la generalización de los resultados a clientes con baja interacción digital
es posible que la tasa de conversión observada esté ligeramente sobreestimada respecto a la población total. Por tanto, los resultados deben interpretarse dentro del segmento de clientes digitalmente activos.

In [17]:
# Ahora vamos a evaluar la duración del experimento
df_events = pd.read_csv("Datos procesados/df_web_events_final.csv")

print(df_events.columns)
print(df_events.shape)


Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time',
       'time_diff', 'time_diff_sec'],
      dtype='object')
(744641, 7)


In [18]:
# convertimos fecha a datetime
df_events["date_time"] = pd.to_datetime(df_events["date_time"])

print(df_events["date_time"].min())
print(df_events["date_time"].max())


2017-03-15 00:03:03
2017-06-20 23:59:57


El experimento se desarrolló entre el 15 de marzo y el 20 de junio de 2017

📆 97 días de experimento
👥 50.500 usuarios en el A/B
📊 744.641 eventos registrados

- Eso es muchísimo volumen.
- En un entorno digital donde:

-El comportamiento ocurre en la misma sesión
-La conversión es inmediata
-No hay ciclo largo de decisión

👉 3 meses es más que suficiente para detectar un efecto real.